In [18]:
import numpy as np
import pandas as pd
import os
import copy
import warnings

warnings.filterwarnings('ignore')

In [19]:
trainPathPick = '../user_data/tmp_data/pickAll.txt'
trainPathDelivery = '../user_data/tmp_data/deliveryAll.txt' 

In [20]:
pick = pd.read_csv(trainPathPick)
delivery = pd.read_csv(trainPathDelivery)

In [21]:
# 获取每个订单后续的取单和送单的候选集合
def getOrderDivide(pickData,actionName):
                  # 0        1     2          3            4            5      6            7             8       9            10          11            12      13              14           15       16        17            18             19        20                21           22      23        24             25         26     27               28            29        30       31                                                
    columnsName2=['订单号','动作','通知时间','可取单时间','承诺送达时间','距离','理论完成时间','实际完成时间','订单1','通知时间1','可取单时间1','承诺送达时间1','距离1','理论完成时间1','实际完成时间1','订单2','通知时间2','可取单时间2','承诺送达时间2','距离2','理论完成时间2','实际完成时间2','骑手等级','骑手速度','天气']
    columnsName3=['订单号','动作','通知时间','可取单时间','承诺送达时间','距离','理论完成时间','实际完成时间','订单1','通知时间1','可取单时间1','承诺送达时间1','距离1','理论完成时间1','实际完成时间1','订单2','通知时间2','可取单时间2','承诺送达时间2','距离2','理论完成时间2','实际完成时间2','订单3','通知时间3','可取单时间3','承诺送达时间3','距离3','理论完成时间3','实际完成时间3','骑手等级','骑手速度','天气']
    
    pickAll = pickData.loc[pickData['动作'] == actionName,:]
    pick2 = pd.DataFrame(data = np.zeros((len(pickAll),len(columnsName2)),dtype = np.int64),columns = columnsName2) # 存放两个可选单
    pick3 = pd.DataFrame(data = np.zeros((len(pickAll),len(columnsName3)),dtype = np.int64),columns= columnsName3 ) # 存放三个可选单
    
    
    pickOrder = pickAll['订单号']
    pickOrder.drop_duplicates(inplace =True)
    #print(pickOrder)
    pickAll.reset_index(inplace =True,drop = True)
    index1 =0
    index2 =0
    index3 =0
    for i in range(len(pickOrder)):
        order = pickOrder.iloc[i]
        pickOne = pickAll.loc[pickAll['订单号']== order,:]
        #print(pickOne)
        if len(pickOne) ==2:
            
            pick2.loc[index1,'订单号'] = order
            pick2.loc[index1,'动作'] = actionName
            pick2.loc[index1,'通知时间'] = pickOne.iloc[0,3]
            pick2.loc[index1,'可取单时间'] = pickOne.iloc[0,4]
            pick2.loc[index1,'承诺送达时间'] = pickOne.iloc[0,5]
            pick2.loc[index1,'距离'] = pickOne.iloc[0,9]
            pick2.loc[index1,'理论完成时间'] = int(pickOne.iloc[0,9] /pickOne.iloc[0,11] + pickOne.iloc[0,6])
            pick2.loc[index1,'实际完成时间'] = pickOne.iloc[0,2]
            pick2.loc[index1,'骑手等级'] = pickOne.iloc[0,10]
            pick2.loc[index1,'骑手速度'] = pickOne.iloc[0,11]
            pick2.loc[index1,'天气'] = pickOne.iloc[0,23]
            for j in range(2):
                columnIndex = 8+j*7
                pick2.iloc[index1,columnIndex] = pickOne.iloc[j,13]
                pick2.iloc[index1,columnIndex+1] = pickOne.iloc[j,20]
                pick2.iloc[index1,columnIndex+2] = pickOne.iloc[j,21]
                pick2.iloc[index1,columnIndex+3] = pickOne.iloc[j,22]
                pick2.iloc[index1,columnIndex+4] = pickOne.iloc[j,18]
                pick2.iloc[index1,columnIndex+5] = pickOne.iloc[j,19]
                pick2.iloc[index1,columnIndex+6] = pickOne.iloc[j,15]
            index1 = index1+1
        elif len(pickOne) == 3:
            pick3.loc[index2,'订单号'] = order
            pick3.loc[index2,'动作'] = actionName
            pick3.loc[index2,'通知时间'] = pickOne.iloc[0,3]
            pick3.loc[index2,'可取单时间'] = pickOne.iloc[0,4]
            pick3.loc[index2,'承诺送达时间'] = pickOne.iloc[0,5]
            pick3.loc[index2,'距离'] = pickOne.iloc[0,9]
            pick3.loc[index2,'理论完成时间'] = int(pickOne.iloc[0,9] /pickOne.iloc[0,11] + pickOne.iloc[0,6])
            pick3.loc[index2,'实际完成时间'] = pickOne.iloc[0,2]
            pick3.loc[index2,'骑手等级'] = pickOne.iloc[0,10]
            pick3.loc[index2,'骑手速度'] = pickOne.iloc[0,11]
            pick3.loc[index2,'天气'] = pickOne.iloc[0,23]
            for j in range(3):
                columnIndex = 8+j*7
                pick3.iloc[index2,columnIndex] = pickOne.iloc[j,13]
                pick3.iloc[index2,columnIndex+1] = pickOne.iloc[j,20]
                pick3.iloc[index2,columnIndex+2] = pickOne.iloc[j,21]
                pick3.iloc[index2,columnIndex+3] = pickOne.iloc[j,22]
                pick3.iloc[index2,columnIndex+4] = pickOne.iloc[j,18]
                pick3.iloc[index2,columnIndex+5] = pickOne.iloc[j,19]
                pick3.iloc[index2,columnIndex+6] = pickOne.iloc[j,15]
            index2 = index2+1
      
        
    return pick2,pick3   

In [22]:
# 计算差值，构建模型的输入特征
def Caculate(data2,data3):
    
    data2.drop_duplicates(inplace=True,keep=False)
    data3.drop_duplicates(inplace=True,keep=False)
    data2['1-2理论时间差'] =data2['理论完成时间1'] - data2['理论完成时间2']
    data2['1-2距离差'] = data2['距离1'] - data2['距离2']
    data2['1-2通知时间差'] = data2['通知时间1'] - data2['通知时间2']
    data2['1-2可取单时间差'] = data2['可取单时间1']-data2['可取单时间2']
    data2['1-2承诺时间差'] = data2['承诺送达时间1'] - data2['承诺送达时间2']

    data3['1-2理论时间差'] =data3['理论完成时间1'] - data3['理论完成时间2']
    data3['1-2距离差'] = data3['距离1'] - data3['距离2']
    data3['1-2通知时间差'] = data3['通知时间1'] - data3['通知时间2']
    data3['1-2可取单时间差'] = data3['可取单时间1']-data3['可取单时间2']
    data3['1-2承诺时间差'] = data3['承诺送达时间1'] - data3['承诺送达时间2']

    data3['1-3理论时间差'] =data3['理论完成时间1'] - data3['理论完成时间3']
    data3['1-3距离差'] = data3['距离1'] - data3['距离3']
    data3['1-3通知时间差'] = data3['通知时间1'] - data3['通知时间3']
    data3['1-3可取单时间差'] = data3['可取单时间1']-data3['可取单时间3']
    data3['1-3承诺时间差'] = data3['承诺送达时间1'] - data3['承诺送达时间3']

    data3['2-3理论时间差'] =data3['理论完成时间2'] - data3['理论完成时间3']
    data3['2-3距离差'] = data3['距离2'] - data3['距离3']
    data3['2-3通知时间差'] =data3['通知时间2'] - data3['通知时间3']
    data3['2-3可取单时间差'] = data3['可取单时间2']-data3['可取单时间3']
    data3['2-3承诺时间差'] = data3['承诺送达时间2'] - data3['承诺送达时间3']

    data2['标签'] = (data2['订单号'] - data2['订单1']).apply(lambda x : 0 if x==0 else 1)
    data3['标签'] = (data3['订单号'] - data3['订单1']).apply(lambda x : 0 if x==0 else 1)
    data3_1 = data3.loc[data3['标签'] == 0,:]
    data3_2 = data3.loc[data3['标签'] == 1,:]
    data3_2['标签'] = (data3_2['订单号'] - data3_2['订单2']).apply(lambda x: 1 if x==0 else 2)
    data3 = pd.concat([data3_1,data3_2])
    
    return data2,data3
    


In [23]:
# 同时有2个候选取单和3个候选取单
picktwo,pickthree = getOrderDivide(pick,'PICKUP')
# 同时有2个候选送单和3个候选送单
deliverytwo,deliverythree = getOrderDivide(delivery,'DELIVERY')

picktwo,pickthree = Caculate(picktwo,pickthree)
deliverytwo,deliverythree = Caculate(deliverytwo,deliverythree)


picktwo.to_csv('../user_data/tmp_data/pickTwo.txt',index=False,encoding='utf_8')
pickthree.to_csv('../user_data/tmp_data/pickThree.txt',index=False,encoding='utf_8')
deliverytwo.to_csv('../user_data/tmp_data/deliveryTwo.txt',index=False,encoding='utf_8')
deliverythree.to_csv('../user_data/tmp_data/deliveryThree.txt',index=False,encoding='utf_8')

